<a href="https://colab.research.google.com/github/BrunoFerCam/BotYugioh/blob/main/BotYugioh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pytelegrambotapi
! pip install requests
! pip install random
! pip install re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 3.5 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [15]:
import telebot
import requests
import random
import re

In [22]:
API_TOKEN = '6754784484:AAEZSpJAWx7px3V2rksmIgtU5-DLzp1CrIk'
bot = telebot.TeleBot(API_TOKEN)

In [23]:
def get_all_cards():
    response = requests.get('https://db.ygoprodeck.com/api/v7/cardinfo.php')
    if response.status_code == 200:
        data = response.json()
        return data.get('data', [])
    return []

def get_random_card():
    cards = get_all_cards()
    if cards:
        card = random.choice(cards)
        return format_card_info(card)
    return "Failed to retrieve card information."

def get_card_by_context(context):
    cards = get_all_cards()
    filtered_cards = []

    keywords = {
    # Attributes
    'fire': {'attribute': 'FIRE'},
    'water': {'attribute': 'WATER'},
    'earth': {'attribute': 'EARTH'},
    'wind': {'attribute': 'WIND'},
    'light': {'attribute': 'LIGHT'},
    'dark': {'attribute': 'DARK'},
    'divine': {'attribute': 'DIVINE'},

    # Monster Types
    'ritual': {'type': 'Ritual'},
    'fusion': {'type': 'Fusion'},
    'synchro': {'type': 'Synchro'},
    'xyz': {'type': 'XYZ'},
    'link': {'type': 'Link'},
    'pendulum': {'type': 'Pendulum'},
    'tuner': {'type': 'Tuner'},
    'normal': {'type': 'Normal'},
    'effect': {'type': 'Effect'},
    'tuner': {'type': 'Tuner'},
    'gemini': {'type': 'Gemini'},
    'union': {'type': 'Union'},
    'toon': {'type': 'Toon'},

    # Spell Types
    'quick spell': {'type': 'Spell Card', 'race': 'Quick-Play'},
    'field spell': {'type': 'Spell Card', 'race': 'Field'},
    'equip spell': {'type': 'Spell Card', 'race': 'Equip'},
    'normal spell': {'type': 'Spell Card', 'race': 'Normal'},
    'quick-play spell': {'type': 'Spell Card', 'race': 'Quick-Play'},
    'continuous spell': {'type': 'Spell Card', 'race': 'Continuous'},

    # Trap Types
    'counter trap': {'type': 'Trap Card', 'race': 'Counter'},
    'normal trap': {'type': 'Trap Card', 'race': 'Normal'},
    'continuous trap': {'type': 'Trap Card', 'race': 'Continuous'},
    'trap': {'type': 'Trap Card'},

    # Races
    'aqua': {'race': 'Aqua'},
    'beast-warrior': {'race': 'Beast-Warrior'},
    'cyberse': {'race': 'Cyberse'},
    'dinosaur': {'race': 'Dinosaur'},
    'divine-beast': {'race': 'Divine-Beast'},
    'dragon': {'race': 'Dragon'},
    'fairy': {'race': 'Fairy'},
    'fiend': {'race': 'Fiend'},
    'fish': {'race': 'Fish'},
    'illusion': {'race': 'Illusion'},
    'insect': {'race': 'Insect'},
    'machine': {'race': 'Machine'},
    'plant': {'race': 'Plant'},
    'psychic': {'race': 'Psychic'},
    'pyro': {'race': 'Pyro'},
    'reptile': {'race': 'Reptile'},
    'rock': {'race': 'Rock'},
    'sea serpent': {'race': 'Sea Serpent'},
    'spellcaster': {'race': 'Spellcaster'},
    'thunder': {'race': 'Thunder'},
    'warrior': {'race': 'Warrior'},
    'winged beast': {'race': 'Winged Beast'},
    'wyrm': {'race': 'Wyrm'},
    'zombie': {'race': 'Zombie'},
    'beast': {'race': 'Beast'},
    'warrior': {'race': 'Warrior'},
    }


    context_words = context.split()
    found_keyword = False
    level = None
    rank = None

    # Detect and extract level or rank from context
    for i, word in enumerate(context_words):
        if word.lower() == 'level' and i + 1 < len(context_words):
            try:
                level = int(context_words[i + 1])
            except ValueError:
                level = None
        if word.lower() == 'rank' and i + 1 < len(context_words):
            try:
                rank = int(context_words[i + 1])
            except ValueError:
                rank = None

    for card in cards:
        match = True
        if level is not None:
            if 'XYZ' in card['type'] or 'Link' in card['type']:
                continue
            if 'level' not in card or card['level'] != level:
                match = False
        if rank is not None:
            if 'XYZ' not in card['type']:
                continue
            if 'level' not in card or card['level'] != rank:
                match = False

        for word in context_words:
            word_lower = word.lower()
            if word_lower in keywords:
                found_keyword = True
                properties = keywords[word_lower]
                for prop, value in properties.items():
                    if prop == 'type':
                        if value not in card['type']:
                            match = False
                            break
                    else:
                        if card.get(prop) != value:
                            match = False
                            break
        if match:
            filtered_cards.append(card)

    if (level is not None or rank is not None or found_keyword) and filtered_cards:
        card = random.choice(filtered_cards)
        return format_card_info(card)
    else:
        return "No card was found."
        return None

def format_card_info(card):
    card_info = (
        f"**Name:** {card['name']}\n"
        f"**Type:** {card['type']}\n"
    )
    if 'Monster' in card['type']:
        card_info += f"**Attribute:** {card['attribute']}\n"
        card_info += f"**ATK:** {card['atk']}\n"
        if 'Link' in card['type']:
            card_info += f"**Link Rating:** {card['linkval']}\n"
            card_info += f"**Link Arrows:** {', '.join(card['linkmarkers'])}\n"
        else:
            card_info += f"**DEF:** {card['def']}\n"
            if 'XYZ' in card['type']:
              card_info += f"**Rank:** {card['level']}\n"
            else:
              card_info += f"**Level:** {card['level']}\n"
        if 'Pendulum' in card['type']:
            card_info += f"**Pendulum Scale:** {card['scale']}\n"
        card_info += f"**Race:** {card['race']}\n"
        card_info += f"**Description:** {card['desc']}\n"
    elif 'Spell' in card['type'] or 'Trap' in card['type']:
        card_info += (
            f"**Card Type:** {card['race']}\n"
            f"**Description:** {card['desc']}\n"
        )
    if 'card_images' in card and len(card['card_images']) > 0:
        card_info += f"**Image URL:** {card['card_images'][0]['image_url']}\n"
    return card_info

def send_help_message(chat_id):
    help_message = (
        "Welcome to the Yu-Gi-Oh card bot!\n\n"
        "Here are the available commands:\n"
        "/help - Show this help message\n"
        "/random - Get a random Yu-Gi-Oh card\n"
        "/card <keywords> - Search for a card with specific keywords\n\n"
        "You can use attributes, types, and races as keywords. For example:\n"
        "/card fire dragon\n"
        "/card synchro tuner\n"
        "/card field spell"
    )
    bot.send_message(chat_id, help_message)

@bot.message_handler(commands=['start', 'help'])
def handle_help(message):
    send_help_message(message.chat.id)

@bot.message_handler(commands=['random'])
def handle_random(message):
    card_info = get_random_card()
    bot.reply_to(message, card_info, parse_mode='Markdown')

@bot.message_handler(commands=['card'])
def handle_carddb(message):
    user_message = message.text[len('/card '):].strip()
    if user_message:
        card_info = get_card_by_context(user_message)
        if card_info:
            bot.reply_to(message, card_info, parse_mode='Markdown')
        else:
            bot.reply_to(message, "No cards found matching the specified context.", parse_mode='Markdown')
    else:
        bot.reply_to(message, "Please provide keywords to search for a card. Use /help for more information.", parse_mode='Markdown')


In [ ]:
bot.polling()